Loading OpenFF 

In [1]:
import numpy as np
from pathlib import Path
from typing import Any, Callable, Optional, Iterable

from openff.toolkit import ForceField
from openforcefields import openforcefields

ff_dir = openforcefields.resource_filename('openforcefields', 'offxml')
ff_dir = Path(ff_dir)

ff_path = ff_dir / 'openff-2.0.0.offxml'
forcefield = ForceField(ff_path)

## Loading Molecule

In [3]:
from openff.toolkit import Molecule, Topology
from openmm.unit import angstrom, nanometer

STRUCT_DIR = Path('structures')
struct_path = STRUCT_DIR / 'SPW_1m_JC.pdb'

offtop = Topology.from_pdb(str(struct_path))
mols = [mol for mol in offtop.molecules]

In [11]:
# setting unit cell size to specify box volume for PBCs
exclusion = 0 * nanometer # how far to extend the unit cell beyond the molecules boundaries

mol_bbox = offtop.get_positions().ptp(axis=0) # NOTe : OpenFF conformers have coordinates in OpenFF units (NOT OpenMM units)
mol_bbox = mol_bbox.magnitude * nanometer # convert to OpenMM units to simplify combination

box_size = mol_bbox + 2*exclusion # x2 factor accounts for exclusion on either side of box along each axis
offtop.box_vectors = box_size.in_units_of(angstrom)

print(box_size)

[ 4.8057  4.8031 14.4017] nm


## Estimate number of particles needed by physical parameters

In [5]:
from openmm.unit import centimeter, nanometer, angstrom
from openmm.unit import mole, gram
from openmm.unit import AVOGADRO_CONSTANT_NA

from functools import reduce
from operator import mul
from math import ceil

def product(vals : Iterable) -> float:
    '''Takes a collection of numbers and multiplies then all together'''
    return reduce(mul, vals)

def get_number_density(target_density : float, Mw : float) -> float:
    '''Takes a macroscopic density and converts it to a particule number per microscale volume'''
    number_density = (target_density / Mw)
    number_density *= AVOGADRO_CONSTANT_NA # convert from mass to moles

    return number_density.in_units_of(nanometer**-3)
def get_ion_density(concentration : float) -> float:
    '''Takes a macroscopic density and converts it to a particule number per microscale volume'''
    ion_density = (concentration)
    ion_density *= AVOGADRO_CONSTANT_NA # convert from mass to moles

    return ion_density


In [6]:
box_volume = (product(box_size)*1e-24)
conc_ions=1
conc_water=55.5
ions=get_ion_density(conc_ions)
waters=get_ion_density(conc_water)
num_ions=(ions*box_volume)
num_waters=(waters*box_volume)
print('Na:',num_ions,'Cl:',num_ions,'Waters:',num_waters)

Na: 200.19026161896167 nm**3/mol Cl: 200.19026161896167 nm**3/mol Waters: 11110.559519852372 nm**3/mol


## Visualizing molecules in IDE

In [14]:
# mol = next(offtop.molecules)
# mol # calls iPython wrapper under the hood

In [15]:
# mol.visualize(backend='nglview') # this does exactly the same thing

In [16]:
# print(mol) # this gives more info (but is NOT the same thing)

OpenMM Simulation

In [12]:

from openff.interchange import Interchange

from openmm import Integrator
from openmm.openmm import Force
from openmm.app import Simulation

from openmm.unit import nanometer
from openff.units import unit as offunit


# helper function for setting up simulation boilerplate
def create_simulation(interchange : Interchange, integrator : Integrator, forces : Optional[Iterable[Force]]=None) -> Simulation:
    '''Specifies configuration for an OpenMM Simulation - Interchange load alows many routes for creation'''
    openmm_sys = interchange.to_openmm(combine_nonbonded_forces=True)
    openmm_top = interchange.topology.to_openmm()
    openmm_pos = interchange.positions.m_as(offunit.nanometer) * nanometer
    if forces: # deliberately sparse to handle both Nonetype and empty list
        for force in forces:
            openmm_sys.addForce(force)
    
    simulation = Simulation(openmm_top, openmm_sys, integrator)
    simulation.context.setPositions(openmm_pos)

    return simulation

In [13]:
from openmm import LangevinMiddleIntegrator
from openmm.openmm import MonteCarloBarostat

from openmm.unit import femtosecond, picosecond
from openmm.unit import kelvin, atmosphere

timestep = 2 * femtosecond
temp = 300 * kelvin
friction = 1 * picosecond**-1 

In [ ]:
# actual creation using SMIRNOFF and OpenMM
interchange = Interchange.from_smirnoff(forcefield, topology=offtop)
sim = create_simulation(interchange=interchange, integrator=integrator, forces=[barostat])

In [ ]:
# Adding Reporters to allow simulation to output data and trajectories
from openmm.app import PDBReporter, DCDReporter, StateDataReporter, CheckpointReporter 

SIM_DIR = Path('simulations/saltbox_equil/')
record_freq = 100 # how many timesteps between data output

traj_path  = SIM_DIR / 'saltbox.dcd'
check_path = SIM_DIR / 'saltbox.chk'
data_path  = SIM_DIR / 'saltbox.csv'

reported_state_data = { # physical parameters that OpenMM should (or shouldn't) compute
    "step": True,
    "time": True,
    "potentialEnergy": True,
    "kineticEnergy": True,
    "totalEnergy": True,
    "temperature": True,
    "volume": True,
    "density": True,
    "progress": False,
    "remainingTime": False,
    "speed": True,
    "elapsedTime": False
}

traj_rep  = DCDReporter(file=str(traj_path), reportInterval=record_freq)  # save frames at the specified interval
check_rep = CheckpointReporter(str(check_path), reportInterval=record_freq)
state_rep = StateDataReporter(str(data_path), reportInterval=record_freq, **reported_state_data)

reporters = (traj_rep, check_rep, state_rep)
for rep in reporters:
    sim.reporters.append(rep) # add any desired reporters to simulation for tracking

Energy-minimization and simulation run

In [ ]:
integ_time = 50 * picosecond
n_steps = integ_time / timestep # number of integration steps to carry out simulation

sim.minimizeEnergy()
sim.step(n_steps)

### Extracting last frame of trajectory, analyzing data oer trajectory

In [ ]:
import mdtraj as mdt

traj = mdt.load(traj_path, top=struct_path, stride=1)
last_frame = traj[-1]
equil_path  = SIM_DIR / 'saltbox_equil.pdb'    

last_frame.save(equil_path)

In [ ]:
import pandas as pd

df = pd.read_csv(data_path)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df['Box Volume (nm^3)'])